In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions


# Configure webdriver options
options = FirefoxOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Set up webdriver
driver = webdriver.Firefox(options=options)
driver.get("https://divar.ir/s/tehran/car?q=%D9%BE%DA%98%D9%88%20206%20%D8%AA%DB%8C%D9%BE%202")

# Get scroll height
ads = []

last_height = driver.execute_script("return document.body.scrollHeight")
TotalTime = 0
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    if TotalTime > 40:
        break

    last_height = new_height
    TotalTime += 1
    anchors = driver.find_elements(By.TAG_NAME, 'a')
    for anchor in anchors:
        href = anchor.get_attribute("href")
        ads.append(href)

driver.quit()

# Print the ads
print(ads)


In [ ]:
filtered_ads = [href for href in ads if "-206" in href]
print(len(filtered_ads))
for ad in filtered_ads:
    print(ad)

In [ ]:
rows=[]

# Configure webdriver options
option = FirefoxOptions()
option.add_argument("--headless")
option.add_argument("--no-sandbox")
option.add_argument("--disable-dev-shm-usage")

driver = webdriver.Firefox(options=option)

for ad in filtered_ads:
    
    try:
        time.sleep(2)
        
        driver.get(ad)

        sookht = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[3]/div[2]/p')
        if (len(sookht) > 0):
            sookht = sookht[0].text
    #     print(sookht)

        motor_state = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[4]/div[2]/p')
        if (len(motor_state) > 0):
            motor_state = motor_state[0].text
    #     print(motor_state)

        shasi_state = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[5]/div[2]/p')
        if (len(shasi_state) > 0):
            shasi_state = shasi_state[0].text
    #     print(shasi_state)

        badane_state = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[6]/div[2]/p')
        if (len(badane_state) > 0):
            badane_state = badane_state[0].text
    #     print(badane_state)

        bimeh = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[7]/div[2]/p')
        if (len(bimeh) > 0):
            bimeh = bimeh[0].text
    #     print(bimeh)

        gearbox = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[8]/div[2]/p')
        if (len(gearbox) > 0):
            gearbox = gearbox[0].text
    #     print(gearbox)

        cost = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[9]/div[2]/p')
        if (len(cost) > 0):
            cost = cost[0].text
    #     print(cost)

        model = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[1]/div[2]/span[2]')
        if (len(model) > 0):
            model = model[0].text
    #     print(model)

        karkard = driver.find_elements('xpath', '/html/body/div[1]/div[2]/div/div/div[1]/div[4]/div[1]/div[1]/span[2]')
        if (len(karkard) > 0):
            karkard = karkard[0].text
    #     print(karkard)

        row = {
            'نوع سوخت':  sookht,
            'وضعیت موتور': motor_state,
            'وضعیت شاسی‌ها': shasi_state,
            'وضعیت بدنه': badane_state,
            'مهلت بیمهٔ شخص ثالث': bimeh,
            'گیربکس': gearbox,
            'مدل': model,
            'کارکرد': karkard,

            'قیمت پایه': cost
        }

        rows.append(row)
        
    except:
        pass
    
    
    # Open a new window
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[0])
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    
driver.quit()
# Create DataSet
df = pd.DataFrame(rows)
df.to_csv('dataset_206.csv')

In [34]:
df = pd.read_csv('dataset_206.csv')

In [35]:
df = df.rename(columns={
    'نوع سوخت': 'fuel_type',
    'وضعیت موتور': 'engine_condition',
    'وضعیت شاسی‌ها': 'chassis_condition',
    'وضعیت بدنه': 'body_condition',
    'مهلت بیمهٔ شخص ثالث': 'third_party_personal_insurance',
    'گیربکس': 'gearbox',
    'مدل': 'model',
    'کارکرد': 'working',
    'قیمت پایه': 'cost'
})

df.drop('Unnamed: 0', axis=1, inplace=True)


In [36]:
df['fuel_type'].explode().unique()

array(['بنزینی', 'فروشی', '[]'], dtype=object)

In [37]:
df = df[df.fuel_type != 'فروشی']
df

,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
0,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۱ ماه,دنده‌ای,۱۳۸۹,۲۱۱٬۰۰۰,۳۰۰٬۰۰۰٬۰۰۰ تومان
1,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,۵ ماه,دنده‌ای,۱۳۸۳,۳۰۰٬۰۰۰,۲۱۰٬۰۰۰٬۰۰۰ تومان
2,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۲ ماه,دنده‌ای,۱۳۹۱,۲۴۹٬۵۰۰,۲۹۰٬۰۰۰٬۰۰۰ تومان
3,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۸ ماه,دنده‌ای,۱۳۸۵,۲۵۸٬۰۰۰,هستم
4,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۰ ماه,دنده‌ای,۱۳۹۴,۱۴۲٬۰۰۰,۳۰۸٬۰۰۰٬۰۰۰ تومان
...,...,...,...,...,...,...,...,...,...
321,بنزینی,سالم,ضربه‌خورده,سالم و پلمپ,رنگ‌شدگی,۱۱ ماه,۱۳۸۱,۳۵۰٬۰۰۰,۱۶۰٬۰۰۰٬۰۰۰ تومان
322,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۰ ماه,دنده‌ای,۱۳۹۶,۱۰۰٬۰۰۰,۳۲۰٬۰۰۰٬۰۰۰ تومان
323,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۲ ماه,دنده‌ای,۱۳۹۶,۱۲۰٬۰۰۰,هستم
324,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۳ ماه,دنده‌ای,۱۳۸۹,۲۹۰٬۰۰۰,۲۵۲٬۰۰۰٬۰۰۰ تومان


In [38]:
df['gearbox'].explode().unique()

array(['دنده\u200cای', '۱ ماه', '۱۱ ماه', '۶ ماه', '۲ ماه',
       '۹۰٬۰۰۰٬۰۰۰ تومان', '۳۸۵٬۰۰۰٬۰۰۰ تومان', '[]', 'هستم',
       '۱۸۰٬۰۰۰٬۰۰۰ تومان', '۵۰۰٬۰۰۰٬۰۰۰ تومان', '۲۴۰٬۰۰۰٬۰۰۰ تومان',
       '۱۶۵٬۰۰۰٬۰۰۰ تومان', '۴۵۲٬۰۰۰٬۰۰۰ تومان', '۳۱۰٬۰۰۰٬۰۰۰ تومان',
       '۱۲ ماه', '۱۰ ماه', '۲۹۸٬۰۰۰٬۰۰۰ تومان', '۸ ماه',
       '۳۲۵٬۰۰۰٬۰۰۰ تومان', '۲۹۰٬۰۰۰٬۰۰۰ تومان', '۲۰۸٬۰۰۰٬۰۰۰ تومان',
       '۳۱۵٬۰۰۰٬۰۰۰ تومان', '۳۹۰٬۰۰۰٬۰۰۰ تومان', '۴۸۰٬۰۰۰٬۰۰۰ تومان',
       '۱۹۰٬۰۰۰٬۰۰۰ تومان', '۳۸۰٬۰۰۰٬۰۰۰ تومان', '۴۹۷٬۵۰۰٬۰۰۰ تومان',
       '۷ ماه'], dtype=object)

In [39]:
df = df[df.gearbox == 'دنده‌ای']
df

,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
0,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۱ ماه,دنده‌ای,۱۳۸۹,۲۱۱٬۰۰۰,۳۰۰٬۰۰۰٬۰۰۰ تومان
1,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,۵ ماه,دنده‌ای,۱۳۸۳,۳۰۰٬۰۰۰,۲۱۰٬۰۰۰٬۰۰۰ تومان
2,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۲ ماه,دنده‌ای,۱۳۹۱,۲۴۹٬۵۰۰,۲۹۰٬۰۰۰٬۰۰۰ تومان
3,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۸ ماه,دنده‌ای,۱۳۸۵,۲۵۸٬۰۰۰,هستم
4,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۰ ماه,دنده‌ای,۱۳۹۴,۱۴۲٬۰۰۰,۳۰۸٬۰۰۰٬۰۰۰ تومان
...,...,...,...,...,...,...,...,...,...
320,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۶ ماه,دنده‌ای,۱۳۹۹,۵۰٬۰۰۰,۴۲۰٬۰۰۰٬۰۰۰ تومان
322,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۰ ماه,دنده‌ای,۱۳۹۶,۱۰۰٬۰۰۰,۳۲۰٬۰۰۰٬۰۰۰ تومان
323,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۲ ماه,دنده‌ای,۱۳۹۶,۱۲۰٬۰۰۰,هستم
324,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۳ ماه,دنده‌ای,۱۳۸۹,۲۹۰٬۰۰۰,۲۵۲٬۰۰۰٬۰۰۰ تومان


In [40]:
df['third_party_personal_insurance'].explode().unique()

array(['۱۱ ماه', '۵ ماه', '۱۲ ماه', '۸ ماه', '۱۰ ماه', '۳ ماه', '۴ ماه',
       '۶ ماه', '۲ ماه', '۹ ماه', '۷ ماه', '۱ ماه'], dtype=object)

In [41]:
len(df['body_condition'].explode().unique())

9

In [42]:
df['chassis_condition'].explode().unique()

array(['سالم و پلمپ', 'رنگ\u200cشده'], dtype=object)

In [43]:
df['engine_condition'].explode().unique()

array(['سالم', 'تعویض شده', 'نیاز به تعمیر'], dtype=object)

In [44]:
df = df[df.cost != 'هستم']

In [45]:
# Function to remove ' تومان' and convert to integer
def remove_toman_and_convert(cost):
    return int(cost.replace(' تومان', '').replace('٬', ''))

# Apply the function to the 'cost' column
df['cost'] = df['cost'].apply(remove_toman_and_convert)

df

<ipython-input-45-8233df0fadc7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost'] = df['cost'].apply(remove_toman_and_convert)


,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
0,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۱ ماه,دنده‌ای,۱۳۸۹,۲۱۱٬۰۰۰,300000000
1,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,۵ ماه,دنده‌ای,۱۳۸۳,۳۰۰٬۰۰۰,210000000
2,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۲ ماه,دنده‌ای,۱۳۹۱,۲۴۹٬۵۰۰,290000000
4,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۰ ماه,دنده‌ای,۱۳۹۴,۱۴۲٬۰۰۰,308000000
5,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۳ ماه,دنده‌ای,۱۳۹۵,۱۰۳٬۰۰۰,350000000
...,...,...,...,...,...,...,...,...,...
319,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,۳ ماه,دنده‌ای,۱۳۸۷,۲۲۵٬۰۰۰,235000000
320,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۶ ماه,دنده‌ای,۱۳۹۹,۵۰٬۰۰۰,420000000
322,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۰ ماه,دنده‌ای,۱۳۹۶,۱۰۰٬۰۰۰,320000000
324,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۳ ماه,دنده‌ای,۱۳۸۹,۲۹۰٬۰۰۰,252000000


In [46]:
df['working'] = df['working'].str.replace('٬', '')  # Remove the thousands separator
df['working'] = df['working'].astype(int)  # Convert the column to integer type
df

<ipython-input-46-339840e5e938>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['working'] = df['working'].str.replace('٬', '')  # Remove the thousands separator
<ipython-input-46-339840e5e938>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['working'] = df['working'].astype(int)  # Convert the column to integer type


,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
0,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۱ ماه,دنده‌ای,۱۳۸۹,211000,300000000
1,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,۵ ماه,دنده‌ای,۱۳۸۳,300000,210000000
2,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۲ ماه,دنده‌ای,۱۳۹۱,249500,290000000
4,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۰ ماه,دنده‌ای,۱۳۹۴,142000,308000000
5,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۳ ماه,دنده‌ای,۱۳۹۵,103000,350000000
...,...,...,...,...,...,...,...,...,...
319,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,۳ ماه,دنده‌ای,۱۳۸۷,225000,235000000
320,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۶ ماه,دنده‌ای,۱۳۹۹,50000,420000000
322,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۱۰ ماه,دنده‌ای,۱۳۹۶,100000,320000000
324,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,۳ ماه,دنده‌ای,۱۳۸۹,290000,252000000


In [47]:
df['model'] = df['model'].astype(int)  # Convert the column to integer type

<ipython-input-47-eb81d86b5718>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['model'] = df['model'].astype(int)  # Convert the column to integer type


In [48]:
df['third_party_personal_insurance'] = df['third_party_personal_insurance'].str.replace(' ماه', '')
df['third_party_personal_insurance'] = df['third_party_personal_insurance'].astype(int)  # Convert the column to integer type
df

<ipython-input-48-77833a9d3a58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['third_party_personal_insurance'] = df['third_party_personal_insurance'].str.replace(' ماه', '')
<ipython-input-48-77833a9d3a58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['third_party_personal_insurance'] = df['third_party_personal_insurance'].astype(int)  # Convert the column to integer type


,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
0,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,11,دنده‌ای,1389,211000,300000000
1,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,5,دنده‌ای,1383,300000,210000000
2,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,12,دنده‌ای,1391,249500,290000000
4,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,10,دنده‌ای,1394,142000,308000000
5,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,3,دنده‌ای,1395,103000,350000000
...,...,...,...,...,...,...,...,...,...
319,بنزینی,سالم,سالم و پلمپ,خط و خش جزیی,3,دنده‌ای,1387,225000,235000000
320,بنزینی,سالم,سالم و پلمپ,سالم و بی‌خط و خش,6,دنده‌ای,1399,50000,420000000
322,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,10,دنده‌ای,1396,100000,320000000
324,بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی,3,دنده‌ای,1389,290000,252000000


In [49]:
df['fuel_type'] = df['fuel_type'].astype('category').cat.codes
df['engine_condition'] = df['engine_condition'].astype('category').cat.codes
df['chassis_condition'] = df['chassis_condition'].astype('category').cat.codes
df['body_condition'] = df['body_condition'].astype('category').cat.codes
df['gearbox'] = df['gearbox'].astype('category').cat.codes

df

<ipython-input-49-498b49cce1f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fuel_type'] = df['fuel_type'].astype('category').cat.codes
<ipython-input-49-498b49cce1f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['engine_condition'] = df['engine_condition'].astype('category').cat.codes
<ipython-input-49-498b49cce1f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
0,0,1,1,7,11,0,1389,211000,300000000
1,0,1,1,2,5,0,1383,300000,210000000
2,0,1,1,4,12,0,1391,249500,290000000
4,0,1,1,7,10,0,1394,142000,308000000
5,0,1,1,7,3,0,1395,103000,350000000
...,...,...,...,...,...,...,...,...,...
319,0,1,1,2,3,0,1387,225000,235000000
320,0,1,1,7,6,0,1399,50000,420000000
322,0,1,1,4,10,0,1396,100000,320000000
324,0,1,1,4,3,0,1389,290000,252000000


In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# scaler = StandardScaler()
# df = scaler.fit_transform(df)

# columns = ['fuel_type', 'engine_condition', 'chassis_condition', 'body_condition', 'third_party_personal_insurance', 'gearbox', 'model', 'working', 'cost']
# df = pd.DataFrame(df, columns=columns)

df.describe()


,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
count,214.0,214.000000,214.000000,214.00000,214.000000,214.0,214.000000,214.000000,2.140000e+02
mean,0.0,0.995327,0.995327,4.67757,6.700935,0.0,1391.448598,167130.785047,3.623128e+08
std,0.0,0.068359,0.068359,2.10828,3.186577,0.0,6.421404,107017.549445,7.445886e+08
min,0.0,0.000000,0.000000,0.00000,1.000000,0.0,1381.000000,0.000000,1.111111e+07
25%,0.0,1.000000,1.000000,3.00000,4.000000,0.0,1385.000000,67500.000000,2.350000e+08
50%,0.0,1.000000,1.000000,4.00000,6.000000,0.0,1390.000000,167250.000000,2.940000e+08
75%,0.0,1.000000,1.000000,7.00000,9.000000,0.0,1398.000000,249875.000000,3.965000e+08
max,0.0,1.000000,1.000000,8.00000,12.000000,0.0,1401.000000,420000.000000,1.111111e+10


In [57]:
# remove outliers

df = df[df['cost'] > 1.5e08]
df = df[df['cost'] < 4e08]

df = df[df['model'] > 1380]

df.describe()

,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
count,158.0,158.000000,158.000000,158.000000,158.000000,158.0,158.000000,158.000000,1.580000e+02
mean,0.0,0.993671,0.993671,4.189873,6.664557,0.0,1388.835443,210902.455696,2.736171e+08
std,0.0,0.079556,0.079556,1.984500,3.266527,0.0,5.272040,85988.739572,6.586862e+07
min,0.0,0.000000,0.000000,0.000000,1.000000,0.0,1381.000000,25000.000000,1.550000e+08
25%,0.0,1.000000,1.000000,2.000000,4.000000,0.0,1384.000000,142000.000000,2.200000e+08
50%,0.0,1.000000,1.000000,4.000000,6.000000,0.0,1388.000000,206250.000000,2.625000e+08
75%,0.0,1.000000,1.000000,7.000000,9.000000,0.0,1394.000000,270000.000000,3.250000e+08
max,0.0,1.000000,1.000000,8.000000,12.000000,0.0,1400.000000,420000.000000,3.970000e+08


In [58]:
df

,fuel_type,engine_condition,chassis_condition,body_condition,third_party_personal_insurance,gearbox,model,working,cost
0,0,1,1,7,11,0,1389,211000,300000000
1,0,1,1,2,5,0,1383,300000,210000000
2,0,1,1,4,12,0,1391,249500,290000000
4,0,1,1,7,10,0,1394,142000,308000000
5,0,1,1,7,3,0,1395,103000,350000000
...,...,...,...,...,...,...,...,...,...
316,0,1,1,3,7,0,1382,363000,185000000
317,0,1,1,2,2,0,1383,242000,240000000
319,0,1,1,2,3,0,1387,225000,235000000
322,0,1,1,4,10,0,1396,100000,320000000


In [59]:
scaler = StandardScaler()

X, y = df.drop('cost', axis=1), df['cost']
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [60]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np

clf = SVR()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
# print(y_test)
# print()
# print(predictions)

print('RMSE: ', np.sqrt(mean_absolute_error(predictions, y_test)))

RMSE:  7572.564745485922


In [61]:
# 5

import pickle

filename = "car206_model.joblib"
# save model
pickle.dump(clf, open(filename, "wb"))
    

In [62]:
import numpy as np

def car_price_predict(features):
    # load model
    filename = "car206_model.joblib"
    loaded_model = pickle.load(open(filename, "rb"))
    
    return clf.predict(features.reshape(1, -1))


prediction = car_price_predict(X_test[10])
print("true price:" , y_test.iloc[10])
print("predicted price:", prediction)

true price: 258000000
predicted price: [2.55500008e+08]
